In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import roc_auc_score, average_precision_score

from models_sage import HeteroGraphSAGE
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


'cpu'

In [2]:
# Reproducibility control
import torch, random, numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [3]:
SEEDS = [42, 43, 44, 45, 46]


In [4]:
# Notebook-safe paths
ROOT = Path.cwd().parent

DATA_DIR = ROOT / "data" / "data_cleaned"
GRAPH_PATH = ROOT / "outputs" / "data.pt"
OUT_DIR = ROOT / "outputs"
OUT_DIR.mkdir(exist_ok=True)

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("GRAPH_PATH:", GRAPH_PATH)


ROOT: C:\Users\ayish\OneDrive\Documents\circRNA-disease-gnn
DATA_DIR: C:\Users\ayish\OneDrive\Documents\circRNA-disease-gnn\data\data_cleaned
GRAPH_PATH: C:\Users\ayish\OneDrive\Documents\circRNA-disease-gnn\outputs\data.pt


### Load node preserving labelled splits

In [5]:
def load_split(name, le_circ, le_dis):
    df = pd.read_csv(DATA_DIR / name)

    # Encode node names → integer IDs
    circ_ids = le_circ.transform(df["circRNA"].astype(str))
    dis_ids  = le_dis.transform(df["disease"].astype(str))

    # Build edge index: [2, num_edges]
    edges = torch.from_numpy(np.vstack([circ_ids, dis_ids])).long()

    # Labels (already numeric)
    labels = torch.tensor(df["label"].values, dtype=torch.float)

    return edges, labels


In [6]:
encoders = torch.load(
    OUT_DIR / "label_encoders.pt",
    weights_only=False
)

le_circ = encoders["circRNA"]
le_dis  = encoders["disease"]


train_edges, train_labels = load_split("circRNA_disease_train.csv", le_circ, le_dis)
val_edges, val_labels     = load_split("circRNA_disease_val.csv", le_circ, le_dis)
test_edges, test_labels   = load_split("circRNA_disease_test.csv", le_circ, le_dis)

train_edges.shape, train_labels.shape


(torch.Size([2, 929]), torch.Size([929]))

### Move Splits to Device

In [7]:
train_edges, train_labels = train_edges.to(DEVICE), train_labels.to(DEVICE)
val_edges, val_labels     = val_edges.to(DEVICE), val_labels.to(DEVICE)
test_edges, test_labels   = test_edges.to(DEVICE), test_labels.to(DEVICE)


### Load HeteroGraph

In [8]:
print("Loading heterogeneous graph...")
data = torch.load(
    GRAPH_PATH,
    map_location=DEVICE,
    weights_only=False
)

data


Loading heterogeneous graph...


HeteroData(
  circRNA={ x=[828, 6] },
  miRNA={ x=[521, 6] },
  disease={ x=[122, 6] },
  (circRNA, interacts, miRNA)={ edge_index=[2, 896] },
  (miRNA, interacts, disease)={ edge_index=[2, 828] },
  (circRNA, associated, disease)={ edge_index=[2, 985] },
  (circRNA, gip_sim, circRNA)={
    edge_index=[2, 685584],
    edge_weight=[685584],
  },
  (miRNA, gip_sim, miRNA)={
    edge_index=[2, 271441],
    edge_weight=[271441],
  },
  (miRNA, rev_interacts, circRNA)={ edge_index=[2, 896] },
  (disease, rev_interacts, miRNA)={ edge_index=[2, 828] },
  (disease, rev_associated, circRNA)={ edge_index=[2, 985] }
)

### Initialize GraphSAGE model

In [9]:
results = []

for seed in SEEDS:
    print(f"\n==============================")
    print(f"Running experiment with SEED = {seed}")
    print(f"==============================")

    set_seed(seed)

    # ---- model must be re-created AFTER seed ----
    model = HeteroGraphSAGE(
        in_channels=data["circRNA"].x.size(1),
        hidden_channels=64,
        out_channels=64,
        dropout=0.2
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()

    best_val_aupr = 0.0
    best_val_loss = None


    # -------- Training loop --------
    for epoch in range(1, 51):
        model.train()
        optimizer.zero_grad()

        emb = model(data.x_dict, data.edge_index_dict)
        circ_emb, dis_emb = emb["circRNA"], emb["disease"]

        logits = (circ_emb[train_edges[0]] * dis_emb[train_edges[1]]).sum(dim=1)
        loss = loss_fn(logits, train_labels)

        loss.backward()
        optimizer.step()

        # -------- Validation --------
        model.eval()
        with torch.no_grad():
            emb = model(data.x_dict, data.edge_index_dict)
            circ_emb, dis_emb = emb["circRNA"], emb["disease"]

            val_logits = (circ_emb[val_edges[0]] * dis_emb[val_edges[1]]).sum(dim=1)
            val_scores = torch.sigmoid(val_logits).cpu().numpy()
            val_true   = val_labels.cpu().numpy()

            auc = roc_auc_score(val_true, val_scores)
            aupr = average_precision_score(val_true, val_scores)
        print(
        f"Epoch {epoch:03d} | "
        f"Loss {loss.item():.4f} | "
        f"Val AUC {auc:.4f} | "
        f"Val AUPR {aupr:.4f}"
        )

        if aupr > best_val_aupr:
            best_val_aupr = aupr
            best_val_loss = loss.item()
            torch.save(
                model.state_dict(),
                OUT_DIR / f"sage_best_model_seed{seed}.pth"
            )
            print("   → Saved best model")

    # -------- Test evaluation --------
    model.load_state_dict(
        torch.load(OUT_DIR / f"sage_best_model_seed{seed}.pth", map_location=DEVICE)
    )
    model.eval()

    with torch.no_grad():
        emb = model(data.x_dict, data.edge_index_dict)
        circ_emb, dis_emb = emb["circRNA"], emb["disease"]

        test_logits = (circ_emb[test_edges[0]] * dis_emb[test_edges[1]]).sum(dim=1)
        test_scores = torch.sigmoid(test_logits).cpu().numpy()
        test_true   = test_labels.cpu().numpy()

        test_auc  = roc_auc_score(test_true, test_scores)
        test_aupr = average_precision_score(test_true, test_scores)

    print(f"SEED {seed} | Test AUC {test_auc:.4f} | Test AUPR {test_aupr:.4f}")

    results.append((seed, best_val_loss, test_auc, test_aupr))



Running experiment with SEED = 42
3Layer
3Layer
Epoch 001 | Loss 0.6368 | Val AUC 0.7230 | Val AUPR 0.1791
   → Saved best model
3Layer
3Layer
Epoch 002 | Loss 0.5463 | Val AUC 0.8331 | Val AUPR 0.2767
   → Saved best model
3Layer
3Layer
Epoch 003 | Loss 0.4849 | Val AUC 0.8713 | Val AUPR 0.3611
   → Saved best model
3Layer
3Layer
Epoch 004 | Loss 0.4512 | Val AUC 0.8884 | Val AUPR 0.4200
   → Saved best model
3Layer
3Layer
Epoch 005 | Loss 0.4305 | Val AUC 0.8987 | Val AUPR 0.4516
   → Saved best model
3Layer
3Layer
Epoch 006 | Loss 0.4253 | Val AUC 0.9084 | Val AUPR 0.4923
   → Saved best model
3Layer
3Layer
Epoch 007 | Loss 0.4168 | Val AUC 0.9150 | Val AUPR 0.5120
   → Saved best model
3Layer
3Layer
Epoch 008 | Loss 0.4172 | Val AUC 0.9203 | Val AUPR 0.5404
   → Saved best model
3Layer
3Layer
Epoch 009 | Loss 0.4116 | Val AUC 0.9258 | Val AUPR 0.5703
   → Saved best model
3Layer
3Layer
Epoch 010 | Loss 0.4081 | Val AUC 0.9314 | Val AUPR 0.6049
   → Saved best model
3Layer
3Layer
E

In [10]:
results_df = pd.DataFrame(
    results,
    columns=["seed", "best_val_loss", "auc", "aupr"]
)

print("\n===== FINAL RESULTS =====")
print(results_df)
print("\nMean ± Std")
print(
    results_df[["best_val_loss", "auc", "aupr"]]
    .agg(["mean", "std"])
)



===== FINAL RESULTS =====
   seed  best_val_loss       auc      aupr
0    42       0.395115  0.936921  0.571194
1    43       0.387320  0.949537  0.609745
2    44       0.384975  0.918674  0.556816
3    45       0.384783  0.937135  0.593430
4    46       0.389805  0.937295  0.582653

Mean ± Std
      best_val_loss       auc      aupr
mean       0.388400  0.935912  0.582768
std        0.004273  0.011036  0.020303
